In [44]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
iris = datasets.load_iris()
iris_data = iris.data
iris_features = iris.feature_names
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

We assume a mixture model with three Gaussians

In [3]:
print(iris.target_names)
iris_features

['setosa' 'versicolor' 'virginica']


['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [28]:
train_data, test_data, train_labels, test_labels = train_test_split(iris_data,iris.target,test_size=0.33,random_state=24)

In [29]:
train_size = len(train_data)
#test_size = 150 - train_size
test_size = len(test_data)
train_class_cnt = [0,0,0]
test_class_cnt = [0,0,0]
est_prior = [0,0,0]

In [30]:
est_mean = [[0,0,0,0],[0,0,0,0],[0,0,0,0]]
est_covar = [[[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]],[[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]],[[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]]

for i in range(train_size):
    est_mean[train_labels[i]] += train_data[i]
    train_class_cnt[train_labels[i]] += 1

for i in range(train_size):
    mat = np.matrix(train_data[i]-est_mean[train_labels[i]])
    est_covar[train_labels[i]] += np.matmul(mat.transpose(),mat)

for i in range(3):
    est_mean[i] /= train_class_cnt[i]
    est_covar[i] /= train_class_cnt[i]

for i in range(test_size):
    test_class_cnt[test_labels[i]] += 1

for i in range(3):    
    est_prior[i] = (1/train_size)*train_class_cnt[i]

In [31]:
print(train_class_cnt)
print(est_mean)
est_covar

[32, 37, 31]
[array([4.965625, 3.40625 , 1.4625  , 0.25    ]), array([5.97567568, 2.77027027, 4.25945946, 1.31891892]), array([6.5483871 , 2.9483871 , 5.50645161, 1.96451613])]


[matrix([[23695.9209375, 16254.61625  ,  6979.02375  ,  1193.00125  ],
         [16254.61625  , 11150.190625 ,  4787.3846875,   818.3590625],
         [ 6979.02375  ,  4787.3846875,  2055.52     ,   351.3740625],
         [ 1193.00125  ,   818.3590625,   351.3740625,    60.075    ]]),
 matrix([[46278.72432432, 21454.38      , 32987.45162162, 10214.38918919],
         [21454.38      ,  9946.12324324, 15292.69783784,  4735.32054054],
         [32987.45162162, 15292.69783784, 23513.54378378,  7280.84648649],
         [10214.38918919,  4735.32054054,  7280.84648649,  2254.49243243]]),
 matrix([[38593.70258065, 17376.58129032, 32452.90290323, 11578.0283871 ],
         [17376.58129032,  7823.80129032, 14611.72548387,  5212.97774194],
         [32452.90290323, 14611.72548387, 27289.27677419,  9735.82322581],
         [11578.0283871 ,  5212.97774194,  9735.82322581,  3473.4516129 ]])]

In [32]:
common_covar = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]

for i in range(3):
    common_covar += (train_class_cnt[i]/train_size)*est_covar[i]

common_covar

matrix([[36669.8705, 18526.338 , 24499.0446,  7750.2732],
        [18526.338 ,  9673.505 , 11719.8962,  3629.9666],
        [24499.0446, 11719.8962, 17817.4534,  5824.4581],
        [ 7750.2732,  3629.9666,  5824.4581,  1930.1562]])

In [33]:
# equal priors and variances
def makePred_eq_priors_eq_var(trainData,testData,est_mean):
    pred_train = [None]*train_size
    pred_test = [None]*test_size

    for i in range(train_size):
        if i < test_size:
            mat1 = np.matrix(testData[i])
            res = [est_mean[0]*mat1.transpose() - np.linalg.norm(est_mean[0])/2, est_mean[1]*mat1.transpose() - np.linalg.norm(est_mean[1])/2, est_mean[2]*mat1.transpose() - np.linalg.norm(est_mean[2])/2]
            res = res.index(max(list(res)))
            pred_test[i] = res
        mat1 = np.matrix(trainData[i])
        res = [est_mean[0]*mat1.transpose() - np.linalg.norm(est_mean[0])/2, est_mean[1]*mat1.transpose() - np.linalg.norm(est_mean[1])/2, est_mean[2]*mat1.transpose() - np.linalg.norm(est_mean[2])/2]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test

In [34]:
def calcAccuracy(trainLabel_DIST1,count_train,pred_train,testLabel_DIST1,count_test,pred_test):
    train_accuracy = [0,0,0]
    test_accuracy = [0,0,0]
    for i in range(train_size):
        if i < test_size:
            if testLabel_DIST1[i] == pred_test[i]:
                test_accuracy[testLabel_DIST1[i]] += 1
        if trainLabel_DIST1[i] == pred_train[i]:
            train_accuracy[trainLabel_DIST1[i]] += 1

    for i in range(3):
        train_accuracy[i] /= count_train[i]
        test_accuracy[i] /= count_test[i]
    return train_accuracy, test_accuracy

In [35]:
pred_train, pred_test = makePred_eq_priors_eq_var(train_data,test_data,est_mean)
accr1 = calcAccuracy(train_labels,train_class_cnt,pred_train,test_labels,test_class_cnt,pred_test)
accr1

([0.0, 0.0, 1.0], [0.0, 0.0, 1.0])

In [36]:
# estimated priors and variances
# can improve using the associativity of matrix multiplication
def makePred_est_priors_est_var(trainData,testData,est_mean,est_covar,est_prior):
    pred_train = [None]*train_size
    pred_test = [None]*test_size

    for i in range(train_size):
        m_0 = np.matrix(est_mean[0])
        m_1 = np.matrix(est_mean[1])
        m_2 = np.matrix(est_mean[2])
        S_0 = np.matrix(est_covar[0])
        S_1 = np.matrix(est_covar[1])
        S_2 = np.matrix(est_covar[2])
        if i < test_size:
            mat1 = np.matrix(testData[i])
            r1 = np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),mat1.transpose())
            r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),m_0.transpose())
            r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S_0)),m_0.transpose())
            r4 = np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),mat1.transpose())
            r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),m_1.transpose())
            r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S_1)),m_1.transpose())
            r7 = np.matmul(np.matmul(mat1,np.linalg.inv(S_2)),mat1.transpose())
            r8 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_2)),m_2.transpose())
            r9 = np.matmul(np.matmul(m_2,np.linalg.inv(S_2)),m_2.transpose())
            
            res = [-np.log(np.linalg.det(S_0))/2 - (r1-r2+r3)/2 + np.log(est_prior[0]),
                   -np.log(np.linalg.det(S_1))/2 - (r4-r5+r6)/2 + np.log(est_prior[1]),
                   -np.log(np.linalg.det(S_2))/2 - (r7-r8+r9)/2 + np.log(est_prior[2])]
            res = res.index(max(list(res)))
            pred_test[i] = res
            
        mat1 = np.matrix(trainData[i])
        r1 = np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),mat1.transpose())
        r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),m_0.transpose())
        r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S_0)),m_0.transpose())
        r4 = np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),mat1.transpose())
        r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),m_1.transpose())
        r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S_1)),m_1.transpose())
        r7 = np.matmul(np.matmul(mat1,np.linalg.inv(S_2)),mat1.transpose())
        r8 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_2)),m_2.transpose())
        r9 = np.matmul(np.matmul(m_2,np.linalg.inv(S_2)),m_2.transpose())
        
        res = [-np.log(np.linalg.det(S_0))/2 - (r1-r2+r3)/2 + np.log(est_prior[0]),
               -np.log(np.linalg.det(S_1))/2 - (r4-r5+r6)/2 + np.log(est_prior[1]),
               -np.log(np.linalg.det(S_2))/2 - (r7-r8+r9)/2 + np.log(est_prior[2])]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test 

In [37]:
pred_train, pred_test = makePred_est_priors_est_var(train_data,test_data,est_mean,est_covar,est_prior)
accr2 = calcAccuracy(train_labels,train_class_cnt,pred_train,test_labels,test_class_cnt,pred_test)
accr2

([1.0, 0.918918918918919, 0.967741935483871], [1.0, 0.9230769230769231, 1.0])

In [38]:
# using common covariance and estimated priors
def makePred_common_covar_est_prior(trainData,testData,est_mean,com_covar,est_prior):
    pred_train = [None]*train_size
    pred_test = [None]*test_size

    for i in range(train_size):
        m_0 = np.matrix(est_mean[0])
        m_1 = np.matrix(est_mean[1])
        m_2 = np.matrix(est_mean[2])
        S = np.matrix(com_covar)
        if i < test_size:
            mat1 = np.matrix(testData[i])
            r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_0.transpose())
            r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S)),m_0.transpose())
            r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_1.transpose())
            r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S)),m_1.transpose())
            r8 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_2.transpose())
            r9 = np.matmul(np.matmul(m_2,np.linalg.inv(S)),m_2.transpose())
        
            
            res = [-(-r2+r3)/2 + np.log(est_prior[0]),
                   -(-r5+r6)/2 + np.log(est_prior[1]),
                   -(-r8+r9)/2 + np.log(est_prior[2])]
            res = res.index(max(list(res)))
            pred_test[i] = res
            
        mat1 = np.matrix(trainData[i])
        r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_0.transpose())
        r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S)),m_0.transpose())
        r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_1.transpose())
        r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S)),m_1.transpose())
        r8 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_2.transpose())
        r9 = np.matmul(np.matmul(m_2,np.linalg.inv(S)),m_2.transpose())
        res = [-(-r2+r3)/2 + np.log(est_prior[0]),
               -(-r5+r6)/2 + np.log(est_prior[1]),
               -(-r8+r9)/2 + np.log(est_prior[2])]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test  

In [39]:
pred_train, pred_test = makePred_common_covar_est_prior(train_data,test_data,est_mean,common_covar,est_prior)
accr3 = calcAccuracy(train_labels,train_class_cnt,pred_train,test_labels,test_class_cnt,pred_test)
accr3

([0.0, 1.0, 0.0], [0.0, 1.0, 0.0])

In [40]:
# equal variances est. priors
def makePred_est_priors_eq_Allvar(trainData,testData,est_mean,var,est_prior):
    pred_train = [None]*train_size
    pred_test = [None]*test_size
    mean0 = np.matrix(est_mean[0])
    mean1 = np.matrix(est_mean[1])
    mean2 = np.matrix(est_mean[2])

    for i in range(train_size):
        if i < test_size:
            mat1 = np.matrix(testData[i])
            res = [-np.linalg.norm(mat1-mean0)/(2*var) + np.log(est_prior[0]), 
                   -np.linalg.norm(mat1-mean1)/(2*var) + np.log(est_prior[1]),
                   -np.linalg.norm(mat1-mean2)/(2*var) + np.log(est_prior[2])]
            res = res.index(max(list(res)))
            pred_test[i] = res
        mat1 = np.matrix(trainData[i])
        res = [-np.linalg.norm(mat1-mean0)/(2*var)+np.log(est_prior[0]), 
               -np.linalg.norm(mat1-mean1)/(2*var) + np.log(est_prior[1]),
               -np.linalg.norm(mat1-mean2)/(2*var) + np.log(est_prior[2])]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test

In [41]:
s = np.power(np.linalg.det(np.matrix(common_covar)),1/2)
pred_train, pred_test = makePred_est_priors_eq_Allvar(train_data,test_data,est_mean,s,est_prior)
accr4 = calcAccuracy(train_labels,train_class_cnt,pred_train,test_labels,test_class_cnt,pred_test)
accr4

([0.0, 1.0, 0.0], [0.0, 1.0, 0.0])

In [42]:
# Naive-Bayes Classifier
def makePred_naive_bayes(trainData,testData,est_mean,est_covar,est_prior):
    pred_train = [None]*train_size
    pred_test = [None]*test_size

    for i in range(train_size):
        if i < test_size:
            res = [0,0,0]
            for j in range(3):
                tmp = 0
                for k in range(4):
                    tmp += np.power((testData[i][k]-est_mean[j][k]),2)/est_covar[k,k]
                res[j] += -tmp/2 + np.log(est_prior[j])
            res = res.index(max(list(res)))
            pred_test[i] = res
            
        res = [0,0,0]
        for j in range(3):
            tmp = 0
            for k in range(4):
                tmp += np.power((trainData[i][k]-est_mean[j][k]),2)/est_covar[k,k]
            res[j] += -tmp/2 + np.log(est_prior[j])
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test  

In [43]:
pred_train, pred_test = makePred_naive_bayes(train_data,test_data,est_mean,common_covar,est_prior)
accr5 = calcAccuracy(train_labels,train_class_cnt,pred_train,test_labels,test_class_cnt,pred_test)
accr5

([0.0, 1.0, 0.0], [0.0, 1.0, 0.0])

In [61]:
res_table = pd.DataFrame({"naive_bayes_train":accr5[0],
                          "naive_bayes_test": accr5[1],
                          "est_priors_eq_Allvar_train":accr4[0],
                          "est_priors_eq_Allvar_test": accr4[1],
                          "common_covar_est_prior_train":accr3[0],
                          "common_covar_est_prior_test": accr3[1],
                          "est_priors_est_var_train":accr2[0],
                          "est_priors_est_var_test": accr2[1],
                          "eq_priors_eq_var_train":accr1[0],
                          "eq_priors_eq_var_test": accr1[1]})

#res_table.style.set_properties(**{'number-align': 'left'})

res_table

,common_covar_est_prior_test,common_covar_est_prior_train,eq_priors_eq_var_test,eq_priors_eq_var_train,est_priors_eq_Allvar_test,est_priors_eq_Allvar_train,est_priors_est_var_test,est_priors_est_var_train,naive_bayes_test,naive_bayes_train
0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0
1,1.0,1.0,0.0,0.0,1.0,1.0,0.923077,0.918919,1.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,1.000000,0.967742,0.0,0.0
